In [1]:
import pandas as pd
import random
import numpy as np

In [2]:
# read and concat mortality data

def read_concat_mortality():
    import warnings
    warnings.filterwarnings("ignore")
    all_mortality = []
    for year in range(2003, 2016):
        all_mortality.append(pd.read_csv("../00_source_data/US_VitalStatistics/Underlying Cause of Death, " + str(year) + ".txt", error_bad_lines=False, sep="\t",))
    return pd.concat(all_mortality, ignore_index=True)

In [3]:
df = read_concat_mortality()

In [4]:
def clean_mortality(df, State_Abbr):
    df[["County1", "State"]] = df.County.str.split(", ", expand=True)
    df = df[df["State"] == State_Abbr]
    df = df.drop(["Notes", "County", "County Code", "Year Code"], axis=1)
    df = df.rename(columns={"County1": "County"})
    df = df[(df["Drug/Alcohol Induced Cause Code"] == "D1") | (df["Drug/Alcohol Induced Cause Code"] == "D2")
            | (df["Drug/Alcohol Induced Cause Code"] == "D4") | (df["Drug/Alcohol Induced Cause Code"] == "D9")]
    df["Deaths"] = df["Deaths"].astype(float)
    df = df.groupby(["Year", "State", "County"], as_index=False).sum()
    df["Year"] = df["Year"].astype(int)
    return df


In [5]:
def clean_pop(df1,df2,State_ABBR):
    df1 = df1.drop(["Unnamed: 1", 2000, 2001, 2002, "Unnamed: 12", "Unnamed: 13"], axis=1)
    df1 = df1.rename(columns={"Unnamed: 0": "County"})
    df2 = df2.drop(["Census", "Estimates Base", 2016, 2017, 2018, 2019], axis=1)
    df2 = df2.rename(columns={"Unnamed: 0": "County"})
    df1.drop(df1.head(1).index, inplace=True)
    df1.drop(df1.tail(8).index, inplace=True)
    df2.drop(df2.head(1).index, inplace=True)
    df2.drop(df2.tail(5).index, inplace=True)
    df1["County"] = df1["County"].str[1:]
    df2["County"] = df2["County"].str[1:]
    df1 = df1.melt(id_vars=["County"])
    df1 = df1.rename(columns={"variable": "Year", "value": "Population"})
    df1 = df1.groupby(["Year", "County"], as_index=False).sum()
    df2 = df2.melt(id_vars=["County"])
    df2 = df2.rename(columns={"variable": "Year", "value": "Population"})
    df2 = df2.groupby(["Year", "County"], as_index=False).sum()
    df2[["County1", "State"]] = df2.County.str.split(", ", expand=True)
    df2 = df2.drop(["County", "State"], axis=1)
    df2 = df2.rename(columns={"County1": "County"})
    df_concat = pd.concat([df1,df2],ignore_index=True)
    df_concat["State"] = State_ABBR
    return df_concat

In [6]:
# Function that merges population data with mortality data
def merge_mortalitypop(df_mortality, df_pop):
    df_pop["County"] = df_pop["County"].astype("string")
    df_pop["State"] = df_pop["State"].astype("string")
    df_mortality["County"] = df_mortality["County"].astype("string")
    df_mortality["State"] = df_mortality["State"].astype("string")
    merged = df_mortality.merge(df_pop, left_on=["Year", "County"],
                                right_on=["Year", "County"],
                                how="right", indicator=True
                                )
    merged = merged.drop(["State_x"], axis=1)
    merged = merged.rename(columns={"State_y": "State"})
    merged["Deaths"] = merged["Deaths"].apply(
        lambda l: l if not np.isnan(l) else np.random.randint(0, 9)
    )
    merged["Mortality Rate"] = merged["Deaths"] / merged["Population"]
    return merged


# Function that groups dataframe by year, and calculate the total death and total population
def calc_avg_mortality(df1, df2, df3):
    df1 = df1.groupby('Year', as_index=False).sum()[
        ['Year', 'Deaths', 'Population']]
    df2 = df2.groupby('Year', as_index=False).sum()[
        ['Year', 'Deaths', 'Population']]
    df3 = df3.groupby('Year', as_index=False).sum()[
        ['Year', 'Deaths', 'Population']]

    df = pd.concat([df1, df2, df3], ignore_index=True)
    df = df.groupby('Year', as_index=False).sum()
    df['Avg Mortality Rate'] = df['Deaths'] / df['Population']
    return df


In [7]:
fl_mortality = clean_mortality(df, "FL")
fl_pop_pre_policy = pd.read_excel(
    "../00_source_data/Population/FL_and_comparison_states/fl_pop_2000-2009.xls", header=[3])
fl_pop_post_policy = pd.read_excel(
    "../00_source_data/Population/FL_and_comparison_states/fl_pop_2010-2019.xlsx", header=[3])
fl_pop = clean_pop(fl_pop_pre_policy, fl_pop_post_policy, "FL")
fl = merge_mortalitypop(fl_mortality, fl_pop)
fl


,Year,County,Deaths,Population,State,_merge,Mortality Rate
0,2003,Alachua County,11.0,227022.0,FL,both,0.000048
1,2003,Baker County,3.0,23555.0,FL,right_only,0.000127
2,2003,Bay County,21.0,155044.0,FL,both,0.000135
3,2003,Bradford County,8.0,27035.0,FL,right_only,0.000296
4,2003,Brevard County,97.0,504847.0,FL,both,0.000192
...,...,...,...,...,...,...,...
866,2015,Union County,5.0,15265.0,FL,right_only,0.000328
867,2015,Volusia County,98.0,516730.0,FL,both,0.000190
868,2015,Wakulla County,1.0,31523.0,FL,right_only,0.000032
869,2015,Walton County,7.0,63066.0,FL,right_only,0.000111


In [8]:
# fl_mortality = fl.to_csv("../00_source_data/fl_mortality.csv")

In [9]:
tx_mortality = clean_mortality(df, "TX")
tx_pop_pre_policy = pd.read_excel(
    "../00_source_data/Population/TX_and_comparison_states/tx_pop_2000-2009.xls", header=[3])
tx_pop_post_policy = pd.read_excel(
    "../00_source_data/Population/TX_and_comparison_states/tx_pop_2010-2019.xlsx", header=[3])
tx_pop = clean_pop(tx_pop_pre_policy, tx_pop_post_policy, "TX")
tx = merge_mortalitypop(tx_mortality, tx_pop)
tx

,Year,County,Deaths,Population,State,_merge,Mortality Rate
0,2003,Anderson County,0.0,56068.0,TX,right_only,0.000000
1,2003,Andrews County,8.0,12976.0,TX,right_only,0.000617
2,2003,Angelina County,0.0,81510.0,TX,right_only,0.000000
3,2003,Aransas County,3.0,22843.0,TX,right_only,0.000131
4,2003,Archer County,0.0,9013.0,TX,right_only,0.000000
...,...,...,...,...,...,...,...
3297,2015,Wood County,0.0,43117.0,TX,right_only,0.000000
3298,2015,Yoakum County,0.0,8644.0,TX,right_only,0.000000
3299,2015,Young County,7.0,18163.0,TX,right_only,0.000385
3300,2015,Zapata County,5.0,14493.0,TX,right_only,0.000345


In [10]:
# tx_mortality = tx.to_csv("../00_source_data/tx_mortality.csv")


In [11]:
wa_mortality = clean_mortality(df, "WA")
wa_pop_pre_policy = pd.read_excel(
    "../00_source_data/Population/WA_and_comparison_states/wa_pop_2000-2009.xls", header=[3])
wa_pop_post_policy = pd.read_excel(
    "../00_source_data/Population/WA_and_comparison_states/wa_pop_2010-2019.xlsx", header=[3])
wa_pop = clean_pop(wa_pop_pre_policy, wa_pop_post_policy, "WA")
wa = merge_mortalitypop(wa_mortality, wa_pop)
wa

,Year,County,Deaths,Population,State,_merge,Mortality Rate
0,2003,Adams County,7.0,16612.0,WA,right_only,0.000421
1,2003,Asotin County,1.0,20487.0,WA,right_only,0.000049
2,2003,Benton County,10.0,153596.0,WA,both,0.000065
3,2003,Chelan County,0.0,67173.0,WA,right_only,0.000000
4,2003,Clallam County,8.0,66301.0,WA,right_only,0.000121
...,...,...,...,...,...,...,...
502,2015,Wahkiakum County,1.0,4023.0,WA,right_only,0.000249
503,2015,Walla Walla County,8.0,59932.0,WA,right_only,0.000133
504,2015,Whatcom County,23.0,211836.0,WA,both,0.000109
505,2015,Whitman County,3.0,48164.0,WA,right_only,0.000062


In [12]:
# wa_mortality = wa.to_csv("../00_source_data/wa_mortality.csv")